In [1]:
# Import required libraries
from langgraph.graph import START, END, StateGraph
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage, AIMessage
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from typing import TypedDict, Annotated, Optional, List
from pydantic import BaseModel, Field
import operator
import os

print("✅ All imports successful")

✅ All imports successful


In [2]:
# Load API key
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    raise ValueError("OPENAI_API_KEY not found!")

# Initialize LLM
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    api_key=openai_api_key
)

print(f"✅ LLM initialized: {llm.model_name}")

✅ LLM initialized: gpt-4o-mini


In [3]:
# Define Pydantic models for structured outputs

class Plan(BaseModel):
    """Structured plan with steps."""
    task: str
    steps: List[str] = Field(..., description="List of steps to execute")
    reasoning: str = Field(..., description="Why this plan makes sense")

class ExecutionResult(BaseModel):
    """Result from executing a single step."""
    step_number: int
    step_description: str
    result: str
    status: str = Field(default="completed")

class Critique(BaseModel):
    """Structured critique of generated output."""
    clarity: int = Field(..., ge=1, le=5)
    completeness: int = Field(..., ge=1, le=5)
    beginner_friendliness: int = Field(..., ge=1, le=5)
    strengths: List[str]
    weaknesses: List[str]
    needs_refinement: bool
    specific_feedback: str

print("✅ Pydantic models defined")

✅ Pydantic models defined


In [5]:
# Define Hybrid State combining Plan-Execute and Reflection

class HybridState(TypedDict):
    """Combined state for Plan-Execute-Reflection agent."""
    # Input
    task: str
    
    # Plan-Execute phase
    plan: Optional[Plan]
    current_step: int
    execution_results: Annotated[list, operator.add]  # Accumulate results
    
    # Generator phase
    initial_draft: Optional[str]
    
    # Reflection phase
    critique: Optional[Critique]
    refined_output: Optional[str]
    reflection_iterations: int
    
    # Final output
    final_output: Optional[str]
    
    # Metadata
    execution_log: Annotated[list, operator.add]  # Track all actions

print("✅ HybridState defined")

✅ HybridState defined


In [6]:
# System prompts

PLANNER_PROMPT = """You are a strategic planner. Break down the following task into clear, sequential steps.

Task: {task}

Create a step-by-step plan. Return JSON with 'steps' (list of strings) and 'reasoning'."""

EXECUTOR_PROMPT = """You are an executor. Complete this step as thoroughly as possible.

Task: {task}
Step {step_num}: {step_description}

Provide a detailed result for this step."""

GENERATOR_PROMPT = """You are a content generator. Synthesize the following execution results into a cohesive, well-structured summary.

Task: {task}

Execution Results:
{results}

Create a comprehensive draft that integrates all results."""

CRITIC_PROMPT = """You are a quality critic. Evaluate this output.

Task: {task}

Output to Evaluate:
{output}

Evaluate on (1-5 scale):
1. Clarity: Is it clear and well-organized?
2. Completeness: Does it cover all aspects?
3. Beginner-Friendliness: Is it accessible to beginners?

Return JSON with: clarity, completeness, beginner_friendliness (numbers), strengths (list), weaknesses (list), needs_refinement (bool), specific_feedback (string)."""

REFINER_PROMPT = """You are a refiner. Improve this output based on the critique.

Original Output:
{output}

Critique Feedback:
{feedback}

Please refine the output to address the weaknesses, especially improving beginner-friendliness and adding examples where appropriate."""

print("✅ System prompts defined")

✅ System prompts defined


In [7]:
# Helper function to parse JSON from LLM

import json
import re

def extract_json(text: str) -> dict:
    """Extract JSON from LLM response."""
    try:
        # Try direct JSON parse
        return json.loads(text)
    except:
        # Try to find JSON block
        json_match = re.search(r'\{[^{}]*\}', text, re.DOTALL)
        if json_match:
            try:
                return json.loads(json_match.group())
            except:
                pass
    return {}

print("✅ Helper functions defined")

✅ Helper functions defined


In [8]:
# Node: Planner - Creates the execution plan

def planner(state: HybridState) -> dict:
    """Create a plan for the task."""
    print(f"\n{'='*60}")
    print("📋 PLANNER: Creating execution plan...")
    print(f"{'='*60}")
    
    prompt = PLANNER_PROMPT.format(task=state["task"])
    response = llm.invoke([HumanMessage(content=prompt)])
    
    json_data = extract_json(response.content)
    plan = Plan(
        task=state["task"],
        steps=json_data.get("steps", []),
        reasoning=json_data.get("reasoning", "")
    )
    
    print(f"\n✅ Plan created with {len(plan.steps)} steps:")
    for i, step in enumerate(plan.steps, 1):
        print(f"  {i}. {step}")
    print(f"\n📝 Reasoning: {plan.reasoning}")
    
    return {
        "plan": plan,
        "current_step": 1,
        "execution_log": [f"Planned {len(plan.steps)} steps"]
    }

print("✅ Planner node defined")

✅ Planner node defined


In [9]:
# Node: Executor - Executes each step of the plan

def executor(state: HybridState) -> dict:
    """Execute the current step."""
    step_num = state["current_step"]
    plan = state["plan"]
    
    if step_num > len(plan.steps):
        return {"execution_log": ["All steps completed"]}
    
    step_desc = plan.steps[step_num - 1]
    
    print(f"\n⚙️  EXECUTOR: Step {step_num}/{len(plan.steps)}")
    print(f"   Task: {step_desc}")
    
    prompt = EXECUTOR_PROMPT.format(
        task=state["task"],
        step_num=step_num,
        step_description=step_desc
    )
    
    response = llm.invoke([HumanMessage(content=prompt)])
    
    result = ExecutionResult(
        step_number=step_num,
        step_description=step_desc,
        result=response.content
    )
    
    print(f"   ✅ Completed. Result length: {len(response.content)} chars")
    
    return {
        "execution_results": [result],
        "current_step": step_num + 1,
        "execution_log": [f"Step {step_num} completed"]
    }

print("✅ Executor node defined")

✅ Executor node defined


In [10]:
# Conditional node: Should we continue executing?

def should_continue_executing(state: HybridState) -> str:
    """Check if there are more steps to execute."""
    plan = state["plan"]
    current_step = state["current_step"]
    
    if current_step <= len(plan.steps):
        return "executor"
    else:
        return "generator"

print("✅ Conditional node defined")

✅ Conditional node defined


In [11]:
# Node: Generator - Creates initial draft from execution results

def generator(state: HybridState) -> dict:
    """Generate initial output from execution results."""
    print(f"\n{'='*60}")
    print("🔨 GENERATOR: Creating initial draft...")
    print(f"{'='*60}")
    
    # Format execution results
    results_text = "\n".join([
        f"Step {r.step_number}: {r.step_description}\nResult: {r.result[:300]}...\n"
        for r in state["execution_results"]
    ])
    
    prompt = GENERATOR_PROMPT.format(
        task=state["task"],
        results=results_text
    )
    
    response = llm.invoke([HumanMessage(content=prompt)])
    initial_draft = response.content
    
    print(f"\n✅ Initial draft created ({len(initial_draft)} chars)")
    print(f"\n📄 Initial Draft Preview (first 300 chars):\n{initial_draft[:300]}...")
    
    return {
        "initial_draft": initial_draft,
        "execution_log": ["Initial draft generated"]
    }

print("✅ Generator node defined")

✅ Generator node defined


In [12]:
# Node: Critic - Evaluates the generated output

def critic(state: HybridState) -> dict:
    """Critique the generated output."""
    print(f"\n{'='*60}")
    print("🔍 CRITIC: Evaluating output quality...")
    print(f"{'='*60}")
    
    output = state["initial_draft"] if state["initial_draft"] else state["refined_output"]
    
    prompt = CRITIC_PROMPT.format(
        task=state["task"],
        output=output
    )
    
    response = llm.invoke([HumanMessage(content=prompt)])
    
    json_data = extract_json(response.content)
    critique = Critique(
        clarity=json_data.get("clarity", 3),
        completeness=json_data.get("completeness", 3),
        beginner_friendliness=json_data.get("beginner_friendliness", 3),
        strengths=json_data.get("strengths", []),
        weaknesses=json_data.get("weaknesses", []),
        needs_refinement=json_data.get("needs_refinement", False),
        specific_feedback=json_data.get("specific_feedback", "")
    )
    
    print(f"\n📊 Scores: Clarity={critique.clarity}, Completeness={critique.completeness}, Beginner-Friendly={critique.beginner_friendliness}")
    print(f"\n💪 Strengths: {', '.join(critique.strengths)}")
    print(f"\n⚠️  Weaknesses: {', '.join(critique.weaknesses)}")
    print(f"\n💬 Feedback: {critique.specific_feedback}")
    print(f"\n🎯 Needs Refinement: {critique.needs_refinement}")
    
    return {
        "critique": critique,
        "execution_log": [f"Critique: {critique.clarity}/{critique.completeness}/{critique.beginner_friendliness}"]
    }

print("✅ Critic node defined")

✅ Critic node defined


In [13]:
# Conditional node: Should we refine?

def should_refine(state: HybridState) -> str:
    """Check if output needs refinement and if we have iterations left."""
    critique = state["critique"]
    iterations = state["reflection_iterations"]
    
    max_refinement_iterations = 2
    
    if critique.needs_refinement and iterations < max_refinement_iterations:
        print(f"\n🔄 Refining (iteration {iterations + 1}/{max_refinement_iterations})...")
        return "refiner"
    else:
        if iterations >= max_refinement_iterations:
            print(f"\n⏸️  Max refinement iterations reached")
        return "finalizer"

print("✅ Conditional refinement node defined")

✅ Conditional refinement node defined


In [18]:
# Node: Refiner - Refines output based on critique

def refiner(state: HybridState) -> dict:
    """Refine the output based on critique."""
    print(f"\n{'='*60}")
    print("✏️  REFINER: Improving output...")
    print(f"{'='*60}")
    
    current_output = state["refined_output"] if state["refined_output"] else state["initial_draft"]
    critique = state["critique"]
    
    prompt = REFINER_PROMPT.format(
        output=current_output,
        feedback=f"Weaknesses: {', '.join(critique.weaknesses)}\nFeedback: {critique.specific_feedback}"
    )
    
    response = llm.invoke([HumanMessage(content=prompt)])
    refined_output = response.content
    
    print(f"\n✅ Output refined ({len(refined_output)} chars)")
    
    return {
        "refined_output": refined_output,
        "reflection_iterations": state["reflection_iterations"] + 1,
        "execution_log": [f"Refinement iteration {state['reflection_iterations'] + 1} completed"]
    }

print("✅ Refiner node defined")

✅ Refiner node defined


In [19]:
# Node: Finalizer - Returns the final output

def finalizer(state: HybridState) -> dict:
    """Finalize and return the best output."""
    print(f"\n{'='*60}")
    print("🎉 FINALIZER: Process complete!")
    print(f"{'='*60}")
    
    final_output = state["refined_output"] if state["refined_output"] else state["initial_draft"]
    
    return {
        "final_output": final_output,
        "execution_log": ["Process finalized"]
    }

print("✅ Finalizer node defined")

✅ Finalizer node defined


In [20]:
# Build the Graph

graph_builder = StateGraph(HybridState)

# Add nodes
graph_builder.add_node("planner", planner)
graph_builder.add_node("executor", executor)
graph_builder.add_node("generator", generator)
graph_builder.add_node("critic", critic)
graph_builder.add_node("refiner", refiner)
graph_builder.add_node("finalizer", finalizer)

# Add edges
graph_builder.add_edge(START, "planner")
graph_builder.add_edge("planner", "executor")

# Conditional routing for execution loop
graph_builder.add_conditional_edges(
    "executor",
    should_continue_executing,
    {"executor": "executor", "generator": "generator"}
)

# Flow from generator to critic
graph_builder.add_edge("generator", "critic")

# Conditional routing for refinement
graph_builder.add_conditional_edges(
    "critic",
    should_refine,
    {"refiner": "refiner", "finalizer": "finalizer"}
)

# Refiner loops back to critic
graph_builder.add_edge("refiner", "critic")

# Finalizer ends
graph_builder.add_edge("finalizer", END)

# Compile graph with memory
memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)

print("✅ Graph built and compiled successfully")

✅ Graph built and compiled successfully


In [21]:
# Test Scenario: Python Benefits Research

task = "Research the benefits of Python programming, create a summary, and make it beginner-friendly"

initial_state = {
    "task": task,
    "plan": None,
    "current_step": 1,
    "execution_results": [],
    "initial_draft": None,
    "critique": None,
    "refined_output": None,
    "reflection_iterations": 0,
    "final_output": None,
    "execution_log": []
}

config = {"configurable": {"thread_id": "hybrid_agent_1"}}

print(f"\n🚀 Starting Hybrid Agent with task:\n{task}\n")

# Run the graph
result = graph.invoke(initial_state, config)


🚀 Starting Hybrid Agent with task:
Research the benefits of Python programming, create a summary, and make it beginner-friendly


📋 PLANNER: Creating execution plan...

✅ Plan created with 10 steps:
  1. 1. Define the scope of research: Identify specific benefits of Python programming to focus on, such as ease of learning, versatility, community support, and application in various fields.
  2. 2. Gather resources: Collect information from reputable sources such as programming books, online articles, tutorials, and forums that discuss Python programming benefits.
  3. 3. Organize the information: Categorize the benefits into themes (e.g., ease of use, job opportunities, libraries and frameworks, etc.) to create a structured overview.
  4. 4. Summarize the findings: Write a concise summary of each benefit, ensuring that the language is simple and accessible for beginners.
  5. 5. Create examples: Include practical examples or scenarios where Python is beneficial, such as data analysis, 

In [22]:
# Display Final Results

print(f"\n\n{'='*70}")
print("📊 HYBRID AGENT - FINAL REPORT")
print(f"{'='*70}")

print(f"\n📝 PLAN ({len(result['plan'].steps)} steps):")
for i, step in enumerate(result["plan"].steps, 1):
    print(f"  {i}. {step}")

print(f"\n🔨 EXECUTION RESULTS ({len(result['execution_results'])} steps completed):")
for i, exec_result in enumerate(result["execution_results"], 1):
    print(f"\n  Step {i}: {exec_result.step_description}")
    print(f"  Result: {exec_result.result[:200]}...")

print(f"\n📄 INITIAL DRAFT:")
print(result["initial_draft"][:500] + "..." if len(result["initial_draft"]) > 500 else result["initial_draft"])

print(f"\n🔍 CRITIQUE:")
critique = result["critique"]
print(f"  Clarity: {critique.clarity}/5")
print(f"  Completeness: {critique.completeness}/5")
print(f"  Beginner-Friendliness: {critique.beginner_friendliness}/5")
print(f"  Strengths: {', '.join(critique.strengths)}")
print(f"  Weaknesses: {', '.join(critique.weaknesses)}")

if result["refined_output"]:
    print(f"\n✏️  REFINED VERSION (Iteration {result['reflection_iterations']}):")
    print(result["refined_output"][:500] + "..." if len(result["refined_output"]) > 500 else result["refined_output"])

print(f"\n✨ FINAL OUTPUT:")
print(result["final_output"])

print(f"\n📈 METRICS:")
print(f"  Total steps executed: {len(result['execution_results'])}")
print(f"  Refinement iterations: {result['reflection_iterations']}")
print(f"  Final clarity score: {result['critique'].clarity}/5")
print(f"  Final beginner-friendliness: {result['critique'].beginner_friendliness}/5")

print(f"\n{'='*70}")



📊 HYBRID AGENT - FINAL REPORT

📝 PLAN (10 steps):
  1. 1. Define the scope of research: Identify specific benefits of Python programming to focus on, such as ease of learning, versatility, community support, and application in various fields.
  2. 2. Gather resources: Collect information from reputable sources such as programming books, online articles, tutorials, and forums that discuss Python programming benefits.
  3. 3. Organize the information: Categorize the benefits into themes (e.g., ease of use, job opportunities, libraries and frameworks, etc.) to create a structured overview.
  4. 4. Summarize the findings: Write a concise summary of each benefit, ensuring that the language is simple and accessible for beginners.
  5. 5. Create examples: Include practical examples or scenarios where Python is beneficial, such as data analysis, web development, or automation, to illustrate the points.
  6. 6. Review and edit: Go through the summary to ensure clarity, coherence, and that it 